6 1
10 20 10 30 20 50
5 1
3 5 2 6 1
5 1
1 4 2 3 5
5 2
1 4 2 3 5
6 1
2 6 4 1 3 5
6 2
2 6 4 1 3 5
6 3
2 6 4 1 3 5
6 4
2 6 4 1 3 5
6 6
1 1 2 3 2 3

- 시간복잡도 문제로 풀지 못했다.
### 최단거리 index의 의미 해석(18892) - 잘못된 해석
- index 리스트를 거꾸로 보면서 LIS 길이에서 1씩 줄여가며 첫번째로 보이는 index가 가리키는 원소를 출력했다.
- 이렇게 거꾸로가는 iteration 중에 감소하는 순서로 index를 선택하기만 한다면 모든 LIS를 출력할 수 있을 것으로 보인다.
- `DP[x][y]` = index리스트를 거꾸로 보면서, x보다 앞에 있으면서 LIS를 구성할 수 있는, 뒤에서 y번째 숫자인 인덱스들

In [ ]:
import bisect
N, K = map(int, input().split())
DATA = [*map(int, input().split())]
l = [0] * N
LIS = [-float("inf")]
MAX = 0 

for i, n in enumerate(DATA) : 
  if LIS[-1] < n: 
    LIS.append(n)
    l[i] = len(LIS) - 1
    MAX = l[i]
  else:
    l[i] = bisect.bisect_left(LIS, n)
    LIS[l[i]] = n

result = []
def r(i, n, ll) : #i부터 거꾸로 보면서 n을 찾는다.
  # print(i, n, ll)
  if i < 0 : return
  for j in range(i, -1, -1):
    if l[j] != n : continue #찾아야 하는 수
    if DATA[ll[-1]] <= DATA[j] : continue #증가하는 수열이면 안됨
    if n == 1 :
      x = ll+[j]
      lll = []
      for k in x[::-1] :
        lll.append(DATA[k]) #우선순위큐
      result.append(lll)
    r(j-1, n-1, ll+[j])

#재귀
for i in range(len(l)-1, 0, -1) :
  v = l[i]
  if v == MAX :
    r(i-1, v-1, [i])
  
result.sort()
if K > len(result) :
  print(-1)
else :
  print(*result[K-1])

- TLE, 재귀 함수가 시간복잡도 지수적으로 증가.
- [2, 1, 4, 3, 6, 5, ...] 의 입력은 2^(n/2)개 만큼의 LIS가 있다. 따라서 이러한 접근으로는 풀 수 없다.
  - 특히 n이 500인 경우 2^250개가 되므로, 이를 모두 풀기엔 메모리도 부족하게 된다.

### 풀이
- LIS의 개수를 구하는 함수를 만들고, LIS를 재구성 한다 O(N^2 log N)

In [ ]:
import collections
N, K = map(int, input().split())
L = [*map(int, input().split())]

M = [-1] * (N + 1) #메모
def LIS(s) : #L[s]에서 시작하는 IS중 최대 길이를 반환한다.
  ret = M[s + 1]
  if ret != -1 : return ret #항상 S[s]는 있기 때문에 길이는 최소 1이다.

  ret = 1
  for i in range(s + 1, N) :
    if s == -1 or L[s] < L[i] :
      ret = max(ret, LIS(i) + 1)
  return ret
    
def count(s): #S[s]에서 시작하는 LIS의 갯수를 반환한다.
  if LIS(s) == 1: return 1 #L[s]에서 시작하는 LIS가 1이면 자기 자신만 있다.
  ret = M[s + 1]
  if ret != -1 : return ret #메모이제이션

  ret = 0
  for i in range(s + 1, N) :
    if (s == -1 or L[s] < L[i]) and LIS(s) == LIS(i) + 1 :
      ret = min(2000000000, ret + count(i))
  return ret

def reconstruct(s, skip, lis: collections.deque) :
  if s != -1 : lis.appendleft(L[s]) #
  followers = collections.deque()
  for i in range(s + 1, N) :
    if (s == -1 or L[s] < L[i]) and LIS(s) == LIS(i) + 1 :
      followers.appendleft((L[i], i))
  followers = sorted(followers)

  for i in range(len(followers)) :
    idx = followers[i][1]
    cnt = count(idx)
    if cnt <= skip :
      skip -= cnt
    else :
      reconstruct(idx, skip, lis)
      break

x = collections.deque()
reconstruct(-1, K - 1, x)
if x :
  print(" ".join(map(str, reversed(x))))
else :
  print(-1)

- 아무래도 이것도 TLE가 나는 것 같다.

### 아이디어
- 우선 LIS가 $2^{250}$ 개나 나올 수 있는 최악 케이스인 `2 1 4 3 6 5 ...` 의 경우 로컬에서조차 풀리지 않았다.
- LIS()만 돌려도 로컬에서 답이 안나온다. 이를 $O(N \log N)$ 으로 개선해서 총 연산시간을 줄여볼 수 있다. 
  - 여전히 프로그램의 시간복잡도는 $O(N^2 \log N)$ 이겠지만, 개선이 될 수 있다.

In [ ]:
import collections, io, os, bisect
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MAX = 2e9 + 7777
def sol() :
  N, K = map(int, input().split())
  L = [*map(int, input().split())]

  DP = [-1] * N #L[i]에서 시작하는 LIS의 길이
  for s in range(N) :
    bis = [-MAX]
    for num in L[s:] :
      if bis[-1] < num :
        bis.append(num)
      else :
        bis[bisect.bisect_left(bis, num)] = num
    DP[s] = len(bis)-1

  def LIS(s) :
    if s == -1: return DP[0] + 1
    return DP[s]
  # print([LIS(-1)] + DP)

  DP2 = [-1] * (N+1) #L[i]에서 시작하는 LIS의 갯수
  def count_lis(s): #S[s]에서 시작하는 LIS의 갯수를 반환한다.
    if LIS(s) == 1: return 1 #L[s]에서 시작하는 LIS가 1이면 자기 자신만 있다.
    ret = DP2[s+1]
    if ret != -1 : return ret

    ret = 0
    for nxt in range(s + 1, N) :
      if (s == -1 or L[s] < L[nxt]) and LIS(s) == (LIS(nxt) + 1) :
        ret = min(MAX, ret + count_lis(nxt))
    DP2[s+1] = ret
    return ret
  # print([count_lis(-1)] + [count_lis(i) for i in range(N)])

  def reconstruct(s, skip, lis: list) :
    if s != -1 : lis.append(L[s]) #S[s]는 길이가 1이므로 항상 LIS에 포함된다.
    followers = []
    for nxt in range(s + 1, N) :
      if (s == -1 or L[s] < L[nxt]) and LIS(s) == (LIS(nxt) + 1) :
        followers.append((L[nxt], nxt))
    followers.sort()

    for v, idx in followers :
      #이 숫자 뒤에 이어서 만들 수 있는 LIS의 갯수를 계산한다.
      cnt = count_lis(idx)
      if cnt <= skip :
        skip -= cnt
      else :
        reconstruct(idx, skip, lis)
        break

  x = []
  reconstruct(-1, K - 1, x)
  if x :
    print(" ".join(map(str, x)))
  else :
    print(-1)

sol()

In [ ]:
import sys, io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

MAX = 2e9 + 7777
def sol() :
  N, K = map(int, input().split())
  L = [*map(int, input().split())]

  DP = [-1] * (N + 1) #L[i]에서 시작하는 LIS의 길이
  def LIS(s) : #L[s]에서 시작하는 IS중 최대 길이를 반환한다. O(N^2)
    ret = DP[s + 1]
    if ret != -1 : return ret #항상 S[s]는 있기 때문에 길이는 최소 1이다.

    ret = 1
    for i in range(s + 1, N) :
      if s == -1 or L[s] < L[i] :
        ret = max(ret, LIS(i) + 1)
    DP[s+1] = ret
    return ret

  DP2 = [-1] * (N+1) #L[i]에서 시작하는 LIS의 갯수
  def count_lis(s): #S[s]에서 시작하는 LIS의 갯수를 반환한다.
    if LIS(s) == 1: return 1 #L[s]에서 시작하는 LIS가 1이면 자기 자신만 있다.
    if DP2[s+1] != -1 : return DP2[s+1]

    ret = 0
    for nxt in range(s + 1, N) : #s보다 뒤에있는 인덱스 nxt 들을 순회하면서 nxt로부터 만들 수 있는 LIS의 개수를 재귀적으로 센다.
      if (s == -1 or L[s] < L[nxt]) and LIS(s) == LIS(nxt) + 1 :
        ret = min(MAX, ret + count_lis(nxt))
    DP2[s+1] = ret
    return ret
  #L[s]를 시작으로 하는 lis의 다음 s 다음의 index들의 집합을 S라고 할 때, L[s+1:] != S 이기 때문에 count_lis(0) 한번 만으로 모든 경우를 셀 수 없다.

  track = []
  def reconstruct(s, skip) :
    if s != -1 : track.append(L[s]) #
    followers = []
    for nxt in range(s + 1, N) :
      if (s == -1 or L[s] < L[nxt]) and LIS(s) == LIS(nxt) + 1 :
        followers.append((L[nxt], nxt))
    followers.sort()

    for _, idx in followers :
      #L[idx:]부터 만들 수 있는 LIS의 갯수를 계산한다.
      cnt = count_lis(idx)
      if cnt <= skip :
        skip -= cnt
      else :
        reconstruct(idx, skip)
        break

  reconstruct(-1, K - 1)
  sys.stdout.write(" ".join(map(str, track)) + "\n" if track else "-1")

sol()

- 충격과 공포의 포인터 이슈. 참고를 위해 사용한 &int 의 존재를 간과하고 있었다. \
그 포인터에 있는 값을 수정하는 거였는데, Python코드로 옮겨적으면서 그부분을 간과했고, 계속 메모이제이션이 되지 않았다!

### 풀이
- 시간복잡도: $O(N^2 \log N)$
- k번째 답 출력하기가 유형의 문제. 해당 테크닉의 레시피는 다음과 같다.
  1. 정답을 사전순으로 만들며 경우의 수를 세는 완전탐색 알고리즘을 설계한다.
  2. 메모이제이션을 적용해 중복되는 경우를 제거한 DP 알고리즘으로 바꾼다.
  3. 모든 답들을 사전순으로 생성하여 $skip$ 개를 건너뛴 다음의 답을 출력하는 알고리즘을 만든다. \
  이때, $M = \text{(현재 상태로부터 만들 수 있는 답의 수)}$ 을 $skip$ 과 비교하여 다음을 처리한다.
      - $M \leq skip$: $M$ 개의 덥이 모두 원하는 답보다 앞에 있으므로(skip가능하므로) 이들을 건너뛴다. 그리고 $skip := skip - M$ 으로 갱신한다.
      - $M > skip$: 원하는 답이 $M$ 개의 답 중에 있고, 현재 상태에서 $skip$ 번 건너뛴 것이 출력해야할 답이다.\
      정답을 출력하고, 재귀함수로 나머지 부분을 출력한다.

- 레시피에 의하면, M을 구할 수 있어야 하는데, 이는 $L[i:]$ 에서 만들 수 있는 LIS의 개수라고 볼 수 있다.
  - 우선 $L[i:]$ 에서 만들 수 있는 LIS의 길이를 구한다. 이를 $DP[i]$ 에 저장한다.
    - $O(N \log N)$ LIS가 생각날 수 있지만, 전이할 때 구해둔 DP값을 이용할 수 없기 때문에 $O(N^2 \log N)$ 이 돼버릴 것이다.\
    따라서 애초에 $L[i:]$ 에서 만드는 LIS의 길이를 구하는 $O(N^2)$ LIS 알고리즘이 최적임을 알 수 있다. 아마도.
  - $i$ 번째 index에서 만들 수 있는 LIS의 개수를 구하는 함수를 $C(i)$ 라고 하자. $DP$ 를 이용하여 구할 수 있다.\
  $i$ 를 시작으로 하는 lis에서 $i$ 다음에 올 수 있는 원소들의 후보 인덱스를 $j$ 라고하자.
    - $i < j$: $i$ 보다 뒤에있는 원소를 보아야 하므로 당연하다.
    - $L[i] < L[j]$: LIS의 정의에 의해 뒤에있는 원소의 값이 더 커야한다.
    - $\displaystyle DP[i] = DP[j] + 1$: 핵심. $DP[j]$ 는 $DP[i]$ 보다 정확히 1 감소한 값이여야 한다.\
    (증명) 아니라고 해보자. 즉, $DP[i] < DP[j] + 1$ 혹은 $DP[i] > DP[j] + 1$ 일때 lis를 구성할 수 있다고 가정하자.\
    전자의 경우, 더 앞에있는 $DP[i]$ 를 포함해서 만든 lis보다 $DP[j]$ 를 포함하여 구성했을 때의 lis의 길이가 더 길다는 의미이다. \
    애초에 $DP[j]$ 를 포함하면 $i$ 보다 더 뒤에있는 원소들을 가지고 lis를 만들 수 있었다는 의미이므로, 최적해가 아니게된다.\
    후자의 경우, $DP[j]$ 가 아닌 원소를 포함했을 때 더 긴 lis를 만들 수 있다는 의미이다. 따라서 최적해가 아니다.
    - 위 조건을 만족하는 $i$ 다음에 올 수 있는 $j$ 의 집합을 $\text{next}(i)$ 라고 해보자, $C(i)$ 는 다음과 같이 정의할 수 있다.\
    $C(i) = \sum_{j \in \text{next}(i)} C(j)$
- $M$ 를 구하는 함수를 만들었으니, 이제 $skip$ 과 비교하여 문제를 풀 수 있다.
  > $C(i) = M$ 이므로 앞에서부터 한자리씩, 
  - 단, 초기 재귀함수 호출에서 $C(0) \leq skip$ 일 경우, 애초에 $L$ 로 부터 만들 수 있는 $k$ 보다 적다는 의미이므로, -1을 출력한다.

- TODO: 속도 개선O(N^2), iterative

In [ ]:
import sys
input = sys.stdin.readline
N,M = map(int,input().split())
L = list(map(int, input().split()))
L = [int(-1e9)]+L+[int(1e9)]
dp = [1]*(N+2)
dp2 = [-1]*(N+2)
s = []
s.append((L[0],0))
for i in range(1,N+2):
    for j in range(i):
        if L[j] < L[i] and dp[i] < dp[j]+1:
            dp[i] = dp[j]+1
    s.append((L[i], i))
s.sort()
K = max(dp)
ans = []
dp[-1] = K

def rec(p):
    if p == N: return 1
    if dp2[p] != -1: return dp2[p]
    dp2[p] = 0
    for i in range(p+1,N+1):
        if L[p] < L[i] and dp[p]+1==dp[i]:
            dp2[p] = min(1e9,dp2[p]+rec(i))
    return dp2[p]

def f(p, K, s):
    if p == N: return 1
    if p:
        ans.append(L[p])
    new = s.copy()
    for x,y in new:
        if L[p] < L[y] and dp[p]+1 == dp[y]:
            if K < rec(y):
                for i,j in new:
                    if j <= y:
                        s.remove((i,j))
                return f(y,K,s.copy())
            K -= rec(y)
    return 0
N += 1
f(0,M-1,s)
if ans:
    print(*ans)
else:
    print(-1)